# Upcoming Trip Stops, Part 1
Objectives:
- build a list of unique stops that are available from the next earliest trips that pass through the current stop

---
## Import Modules & Data

In [1]:
import time
import numpy as np
import pandas as pd
import numexpr as ne
from datetime import timedelta as td

In [2]:
%load_ext line_profiler

In [3]:
stops = pd.read_feather('data/model/stops.ftr')
schedule = pd.read_feather('data/model/schedule.ftr')

In [1]:
schedule.head(1)

NameError: name 'schedule' is not defined

In [5]:
stops.head(1)

,stop_id,stop_code,stop_name,stop_lat,stop_lon
0,262,662,Danforth Rd at Kennedy Rd,43.714379,-79.260939


In [229]:
df = schedule.copy()
df = df.drop(columns = ['route_short_name', 'route_long_name', 'trip_headsign', 'stop_code', 'stop_name', 'stop_time'])
df = df.sort_values(by = ['stop_time_delta', 'trip_id', 'stop_sequence'])
df['seconds'] = df.stop_time_delta.apply(lambda x: x.total_seconds())

,trip_id,shape_id,stop_sequence,stop_time_delta,stop_id,stop_lat,stop_lon,sec
450018,43003469,887148,1,0 days 03:32:05,8533,43.705192,-79.504228,12725.0
450019,43003469,887148,2,0 days 03:32:35,14963,43.706567,-79.504667,12755.0
450020,43003469,887148,3,0 days 03:33:32,9271,43.709123,-79.505308,12812.0
450021,43003469,887148,4,0 days 03:35:16,5572,43.713764,-79.506522,12916.0
450022,43003469,887148,5,0 days 03:35:47,1126,43.715125,-79.506944,12947.0
...,...,...,...,...,...,...,...,...
365690,43001368,886893,85,1 days 06:40:08,4425,43.670847,-79.295932,110408.0
365691,43001368,886893,86,1 days 06:40:54,15135,43.671493,-79.292818,110454.0
365692,43001368,886893,87,1 days 06:42:10,10371,43.672555,-79.287687,110530.0
365693,43001368,886893,88,1 days 06:42:45,4598,43.673035,-79.285343,110565.0


In [ ]:
df

In [171]:
df.stop_time_delta.to_numpy()

array([68475000000000, 68544000000000, 68566000000000, 68632000000000,
       68713000000000, 68765000000000, 68878000000000, 68949000000000,
       69002000000000, 69063000000000, 69117000000000, 69164000000000,
       69247000000000, 69295000000000, 69326000000000, 69361000000000,
       69383000000000, 69417000000000, 69442000000000, 69483000000000,
       69494000000000, 69531000000000, 69657000000000],
      dtype='timedelta64[ns]')

In [174]:
np.timedelta64(19, 'h')

numpy.timedelta64(19,'h')

In [206]:
%%timeit

select_condition = df.stop_id.values == 917
select_condition

593 µs ± 42.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [208]:
%%timeit

x = df.stop_id.values
select_condition = ne.evaluate('x == 917')
select_condition

196 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [156]:
current_time = td(hours = 19)

In [224]:
df.stop_time_delta.to_numpy()

array([ 12725000000000,  12755000000000,  12812000000000, ...,
       110530000000000, 110565000000000, 110607000000000],
      dtype='timedelta64[ns]')

In [228]:
df.stop_time_delta.total_seconds()

AttributeError: 'Series' object has no attribute 'total_seconds'

In [235]:
%%time

a = df.stop_id.values
x = df.sec.values
y = td(houzzzrs = 19).total_seconds()
select_condition = ne.evaluate('(x > y) & (a == 917)')
select_condition

CPU times: user 8.71 ms, sys: 800 µs, total: 9.51 ms
Wall time: 1.32 ms


array([False, False, False, ..., False, False, False])

In [271]:
%%time

a = df.stop_id.values
x = df.stop_time_delta.to_numpy()
y = np.timedelta64(19, 'h')
select_condition = (x > y) & (a == 917)
select_condition

CPU times: user 3.79 ms, sys: 996 µs, total: 4.79 ms
Wall time: 3.33 ms


array([False, False, False, ..., False, False, False])

In [274]:
%%time
select_condition = df.stop_time_delta.to_numpy() > np.timedelta64(19, 'h')
select_condition

CPU times: user 2.63 ms, sys: 952 µs, total: 3.58 ms
Wall time: 2.18 ms


array([False, False, False, ...,  True,  True,  True])

In [273]:
%%time
x = df.stop_time_delta
y = td(hours = 19)
select_condition = (x > y).values
select_condition

CPU times: user 4.26 ms, sys: 1.07 ms, total: 5.33 ms
Wall time: 3.79 ms


array([False, False, False, ...,  True,  True,  True])

In [147]:
trip_list

array([43005600, 43005602, 43005603, 43005604, 43027158, 43027159,
       43027160, 43027187, 43027188, 43027189, 43027190, 43027191,
       43027192, 43027193, 43027194, 43027195, 43027196, 43027197,
       43027198, 43027199, 43027200, 43027202, 43027203, 43027204,
       43027205, 43027206, 43027207, 43027208, 43027209, 43027210,
       43027211, 43027212, 43027213, 43027214, 43027215, 43027216,
       43027217, 43027218, 43027272, 43027273, 43027274, 43027275,
       43027276, 43027277, 43027278, 43027279, 43027280, 43027281,
       43027282, 43027283, 43027284, 43027285, 43027286, 43027287,
       43027288, 43027289, 43027290, 43029300, 43029301])

In [146]:
select_condition = ((df.stop_id.values == 917) & (df.stop_time_delta > td(hours = 19)).values)
trip_list = np.unique(df[select_condition].trip_id.values)
df = df[df.trip_id.isin(trip_list)]

df2 = df.loc[df.stop_id.values == 917, ['shape_id', 'stop_sequence']].drop_duplicates().set_index('shape_id')
df['target_sequence'] = df.shape_id.map(df2.stop_sequence)
df = df[df.stop_sequence.values > df.target_sequence]
df = df.drop_duplicates(subset = 'stop_id')


df

,trip_id,shape_id,stop_sequence,stop_time_delta,stop_id,stop_lat,stop_lon,target_sequence
1186126,43027160,888274,49,0 days 19:01:15,5191,43.759132,-79.333893,48
1186127,43027160,888274,50,0 days 19:02:24,7238,43.757624,-79.339319,48
1186128,43027160,888274,51,0 days 19:02:46,5738,43.757207,-79.341181,48
1186129,43027160,888274,52,0 days 19:03:52,9834,43.756053,-79.346600,48
1186130,43027160,888274,53,0 days 19:05:13,6330,43.755228,-79.350256,48
1186131,43027160,888274,54,0 days 19:06:05,916,43.754734,-79.352534,48
1186132,43027160,888274,55,0 days 19:07:58,9974,43.753660,-79.357427,48
1186133,43027160,888274,56,0 days 19:09:09,944,43.753001,-79.360766,48
1186134,43027160,888274,57,0 days 19:10:02,1509,43.752639,-79.365268,48
1186135,43027160,888274,58,0 days 19:11:03,9959,43.751639,-79.370302,48


In [46]:
df2 = df.loc[df.stop_id.values == 917, ['shape_id', 'stop_sequence']].drop_duplicates().set_index('shape_id')

In [23]:
df.tail(50)

,trip_id,shape_id,stop_sequence,stop_time_delta,stop_id,stop_lat,stop_lon,select_trip,apply_trip,select_shape,apply_shape,select_sequence,apply_sequence,compare_sequence
1186107,43027160,888274,30,0 days 18:44:50,2799,43.768149,-79.270795,0,43027160,0,888274,0,48,False
1186108,43027160,888274,31,0 days 18:45:12,350,43.767743,-79.272603,0,43027160,0,888274,0,48,False
1186109,43027160,888274,32,0 days 18:46:26,355,43.766455,-79.278682,0,43027160,0,888274,0,48,False
1186110,43027160,888274,33,0 days 18:46:51,2010,43.766017,-79.280760,0,43027160,0,888274,0,48,False
1186111,43027160,888274,34,0 days 18:47:56,1485,43.765185,-79.284336,0,43027160,0,888274,0,48,False
1186112,43027160,888274,35,0 days 18:48:50,8864,43.764459,-79.287449,0,43027160,0,888274,0,48,False
1186113,43027160,888274,36,0 days 18:50:07,4604,43.763409,-79.291907,0,43027160,0,888274,0,48,False
1186114,43027160,888274,37,0 days 18:50:47,6543,43.762790,-79.294575,0,43027160,0,888274,0,48,False
1186115,43027160,888274,38,0 days 18:51:30,8592,43.762102,-79.297487,0,43027160,0,888274,0,48,False
1186116,43027160,888274,39,0 days 18:52:14,7360,43.761429,-79.300415,0,43027160,0,888274,0,48,False


In [7]:
def build_table(df):
    
    select_condition = ((df.stop_id.values == 917) & (df.stop_time_delta > td(hours = 19)).values)
    df['select_trip'] = np.where(select_condition, df.trip_id.values, 0)
    trip_list = np.unique(df.select_trip.values)
    df['apply_trip'] = np.where(np.isin(df.trip_id.values, trip_list), df.trip_id.values, 0)

    df['select_shape'] = np.where(select_condition, df.shape_id.values, 0)
    shape_list = np.unique(df.shape_id.values)
    df['apply_shape'] = np.where(np.isin(df.shape_id.values, shape_list), df.shape_id.values, 0)

    df['select_sequence'] = np.where(select_condition, df.stop_sequence.values, 0)
    df2 = df.loc[select_condition, ['select_shape', 'select_sequence']].drop_duplicates()
    df['apply_sequence'] = df.apply_shape.map(df2.set_index('select_shape')['select_sequence'])

    compare_selection = ((df.apply_trip.values != 0) & (df.stop_sequence.values > df.apply_sequence.values))
    df['compare_sequence'] = np.where(compare_selection, True, False)
    df = df[compare_selection].drop_duplicates(subset = 'stop_id')

    return df

In [18]:
def build_table(df):
    
    select_condition = ((df.stop_id.values == 917) & (df.stop_time_delta > td(hours = 19)).values)
    df['select_trip'] = np.where(select_condition, df.trip_id.values, 0)
    trip_list = np.unique(df.select_trip.values)
    df['apply_trip'] = np.where(np.isin(df.trip_id.values, trip_list), df.trip_id.values, 0)

    df['select_shape'] = np.where(select_condition, df.shape_id.values, 0)
    shape_list = np.unique(df.shape_id.values)
    df['apply_shape'] = np.where(np.isin(df.shape_id.values, shape_list), df.shape_id.values, 0)

    df['select_sequence'] = np.where(select_condition, df.stop_sequence.values, 0)
    df2 = df.loc[select_condition, ['select_shape', 'select_sequence']].drop_duplicates()
    df['apply_sequence'] = df.apply_shape.map(df2.set_index('select_shape')['select_sequence'])

    compare_selection = ((df.apply_trip.values != 0) & (df.stop_sequence.values > df.apply_sequence.values))
    df['compare_sequence'] = np.where(compare_selection, True, False)
    df = df[compare_selection].drop_duplicates(subset = 'stop_id')

    return df

In [267]:
def build_table2(df):
    
    stop_id = df.stop_id.values
    stop_time = df.stop_time_sec.values
    current_time = td(hours = 19).total_seconds()
    select_condition = ne.evaluate('(stop_id == 917) & (stop_time > current_time)')
    
#     select_condition = ((df.stop_id.values == 917) & (df.stop_time_delta > td(hours = 19)).values)
#     trip_list = np.unique(df[select_condition].trip_id.values)
    df = df[df.trip_id.isin(df[select_condition].trip_id)]
#     df = df[np.isin(df.trip_id.values, trip_list)]

    df2 = df.loc[df.stop_id.values == 917, ['shape_id', 'stop_sequence']].drop_duplicates().set_index('shape_id')
    df = df[df.stop_sequence.values > df.shape_id.map(df2.stop_sequence)]
    df = df.drop_duplicates(subset = 'stop_id')

    return df

In [253]:
def build_table3(df):
    
#     stop_id = df.stop_id.values
#     stop_time = df.stop_time_sec.values
#     current_time = td(hours = 19).total_seconds()
#     select_condition = ne.evaluate('(stop_id == 917) & (stop_time > current_time)')
    
    select_condition = ((df.stop_id.values == 917) & (df.stop_time_delta > td(hours = 19)).values)
#     trip_list = np.unique(df[select_condition].trip_id.values)
    df = df[df.trip_id.isin(df[select_condition].trip_id)]
#     df = df[np.isin(df.trip_id.values, trip_list)]

    df2 = df.loc[df.stop_id.values == 917, ['shape_id', 'stop_sequence']].drop_duplicates().set_index('shape_id')
    df = df[df.stop_sequence.values > df.shape_id.map(df2.stop_sequence)]
    df = df.drop_duplicates(subset = 'stop_id')

    return df

In [268]:
df = schedule.copy()
df = df.drop(columns = ['route_short_name', 'route_long_name', 'trip_headsign', 'stop_code', 'stop_name', 'stop_time'])
df = df.sort_values(by = ['stop_time_delta', 'trip_id', 'stop_sequence'])
df['stop_time_sec'] = df.stop_time_delta.apply(lambda x: x.total_seconds())

In [269]:
%timeit build_table2(df)

31.1 ms ± 702 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [258]:
%timeit build_table3(df)

34.4 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


%lprun -f build_table2 build_table2(df)

In [263]:
build_table2(df)

,trip_id,shape_id,stop_sequence,stop_time_delta,stop_id,stop_lat,stop_lon,stop_time_sec
1186126,43027160,888274,49,0 days 19:01:15,5191,43.759132,-79.333893,68475.0
1186127,43027160,888274,50,0 days 19:02:24,7238,43.757624,-79.339319,68544.0
1186128,43027160,888274,51,0 days 19:02:46,5738,43.757207,-79.341181,68566.0
1186129,43027160,888274,52,0 days 19:03:52,9834,43.756053,-79.346600,68632.0
1186130,43027160,888274,53,0 days 19:05:13,6330,43.755228,-79.350256,68713.0
1186131,43027160,888274,54,0 days 19:06:05,916,43.754734,-79.352534,68765.0
1186132,43027160,888274,55,0 days 19:07:58,9974,43.753660,-79.357427,68878.0
1186133,43027160,888274,56,0 days 19:09:09,944,43.753001,-79.360766,68949.0
1186134,43027160,888274,57,0 days 19:10:02,1509,43.752639,-79.365268,69002.0
1186135,43027160,888274,58,0 days 19:11:03,9959,43.751639,-79.370302,69063.0
